# Nigep example using MNIST dataset

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('../src/nigep/*'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from src.nigep import Nigep
from keras.layers import Dense
from keras.models import Sequential
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.optimizers.legacy import RMSprop
from keras.datasets import mnist
from keras.utils import to_categorical


### Load and preprocess MNIST dataset from Keras package

In [2]:
(x_train, y_train), (X_test, y_test) = mnist.load_data()

x_train = x_train / 255
X_test = X_test / 255

x_train = x_train.reshape(x_train.shape[0],x_train.shape[1]*x_train.shape[2])
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1]*X_test.shape[2])

INPUT_DIMS = x_train.shape[1]
OUTPUT_DIMS = 10

y_train_one_hot = to_categorical(y_train, num_classes=OUTPUT_DIMS)

### Build and compile the MLP model

In [3]:
NUM_EPOCHS = 20

model = Sequential()

model.add(Dense(512, activation="sigmoid", input_shape=(INPUT_DIMS,), name="Hidden_layer_1"))
model.add(Dense(128, activation="sigmoid", name="Hidden_layer_2"))

model.add(Dense(OUTPUT_DIMS, activation="softmax", name="output_layer"))

model.build()

model.compile(optimizer=RMSprop(0.001), loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Hidden_layer_1 (Dense)      (None, 512)               401920    
                                                                 
 Hidden_layer_2 (Dense)      (None, 128)               65664     
                                                                 
 output_layer (Dense)        (None, 10)                1290      
                                                                 
Total params: 468874 (1.79 MB)
Trainable params: 468874 (1.79 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Execute Nigep to obtain the Generalization Profile results

In [4]:
lr_reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.1, min_delta=0.001, patience=5, verbose=1)
erl_stopping = EarlyStopping(patience=4, monitor='val_loss', verbose=1)
callbacks = [erl_stopping, lr_reduce]

nigep = Nigep(
    execution_name='mnist',
    x_data=x_train,
    y_data=y_train_one_hot,
    model=model,
    batch_size=128,
    input_shape=INPUT_DIMS,
    class_mode='categorical',
    k_fold_n=5,
    target_names=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'],
    epochs=1,
)

nigep.execute()

Fold: 0 - Training
1350/1350 [==============================] - 3s 2ms/step - loss: 0.4087 - accuracy: 0.8839 - val_loss: 0.1978 - val_accuracy: 0.9427
Fold: 0 - Testing
Fold: 1 - Training
1350/1350 [==============================] - 3s 2ms/step - loss: 0.1920 - accuracy: 0.9416 - val_loss: 0.1306 - val_accuracy: 0.9627
Fold: 1 - Testing
Fold: 2 - Training
1350/1350 [==============================] - 3s 2ms/step - loss: 0.1330 - accuracy: 0.9591 - val_loss: 0.0877 - val_accuracy: 0.9715
Fold: 2 - Testing
Fold: 3 - Training
1350/1350 [==============================] - 3s 2ms/step - loss: 0.1014 - accuracy: 0.9689 - val_loss: 0.0910 - val_accuracy: 0.9731
Fold: 3 - Testing
Fold: 4 - Training
1350/1350 [==============================] - 3s 2ms/step - loss: 0.0824 - accuracy: 0.9752 - val_loss: 0.0851 - val_accuracy: 0.9756
Fold: 4 - Testing
